GROUP MEMBERS :
1.  Aniruddh Shukla 
2. Gaurav Singhal 
3. Himanshi Bajaj 


# with tf.function 

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import traceback
import contextlib
import time
import tensorboard
from datetime import datetime
from tensorflow.keras import datasets,layers, models
from keras import regularizers
tf.config.experimental_run_functions_eagerly(True)

In [123]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
data = tf.data.Dataset.from_tensor_slices(
    (train_images.astype(np.float32) / 255.0, train_labels.astype(np.int32)))
data = data.batch(128)

# note: we batch the test data, but do not shuffle/repeat
test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.astype(np.float32) / 255.0, test_labels.astype(np.int32))).batch(128)

In [0]:
epochs = 10
n_h = 256
model = models.Sequential([
              layers.Conv2D(32, 3, padding= 'same' , activation= 'relu', input_shape = (28, 28, 1) ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(32, 3 , padding= 'same', activation = 'relu',  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.3)),

              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.4)),

              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.5)),

              layers.Flatten(),
              layers.Dense(n_h, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Dense(n_h//2, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.Dense(10)
              ])

#model.build((None, 784))  # optional -- note None for the batch axis!!

opt = tf.optimizers.Adam(learning_rate= 5 * 1e-3)
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
@tf.function
def gradient(img_batch, lbl_batch):
  with tf.GradientTape() as tape:
      logits = model(img_batch, training=True)
        # loss format is generally: first argument targets, second argument outputs
      xent = loss_fn(lbl_batch, logits)

  varis = model.trainable_variables
  grads = tape.gradient(xent, varis)
  opt.apply_gradients(zip(grads, varis))
  return logits,xent

In [128]:
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
total_start = time.time()
for epoch in range(epochs):
  start = time.time()
  tf.print("epoch number :", epoch)
  for step , (img_batch, lbl_batch) in enumerate(data):
    logits,xent = gradient(img_batch, lbl_batch)
    train_acc_metric(lbl_batch, logits)
    
  tf.print("Loss: {} Accuracy: {}".format(xent, train_acc_metric.result()))
  train_acc_metric.reset_states()
  stop = time.time()
  tf.print("took {} seconds\n".format(stop-start))
total_stop = time.time()
tf.print("took {} seconds\n".format(total_stop-total_start))

epoch number : 0
Loss: 0.3688471019268036 Accuracy: 0.8211833238601685
took 29.49846863746643 seconds

epoch number : 1
Loss: 0.33544862270355225 Accuracy: 0.8852999806404114
took 29.558159828186035 seconds

epoch number : 2
Loss: 0.22858427464962006 Accuracy: 0.8987333178520203
took 29.38642954826355 seconds

epoch number : 3
Loss: 0.2160230129957199 Accuracy: 0.9066333174705505
took 29.398242712020874 seconds

epoch number : 4
Loss: 0.26181963086128235 Accuracy: 0.9121833443641663
took 29.44110679626465 seconds

epoch number : 5
Loss: 0.22560228407382965 Accuracy: 0.9173666834831238
took 29.55054473876953 seconds

epoch number : 6
Loss: 0.22394298017024994 Accuracy: 0.9201333522796631
took 29.438296794891357 seconds

epoch number : 7
Loss: 0.1731649786233902 Accuracy: 0.9253000020980835
took 29.403552055358887 seconds

epoch number : 8
Loss: 0.15514110028743744 Accuracy: 0.927133321762085
took 29.555477619171143 seconds

epoch number : 9
Loss: 0.23071520030498505 Accuracy: 0.92984998

In [129]:
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
  test_acc_metric(lbl_batch, model(img_batch))
tf.print("Test acc: {}".format(test_acc_metric.result()))

Test acc: 0.9161999821662903


In [130]:
print(tf.autograph.to_code(gradient.python_function))

def tf__gradient(img_batch, lbl_batch):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('gradient', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        with tf.GradientTape() as tape:
            logits = ag__.converted_call(model, (img_batch,), dict(training=True), fscope)
            xent = ag__.converted_call(loss_fn, (lbl_batch, logits), None, fscope)
        varis = model.trainable_variables
        grads = ag__.converted_call(tape.gradient, (xent, varis), None, fscope)
        ag__.converted_call(opt.apply_gradients, (ag__.converted_call(zip, (grads, varis), None, fscope),), None, fscope)
        try:
            do_return = True
            retval_ = fscope.mark_return_value((logits, xent))
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)



# Without tf.function

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
data = tf.data.Dataset.from_tensor_slices(
    (train_images.astype(np.float32) / 255.0, train_labels.astype(np.int32)))
data = data.batch(128)

# note: we batch the test data, but do not shuffle/repeat
test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.astype(np.float32) / 255.0, test_labels.astype(np.int32))).batch(128)

In [0]:
epochs = 10
n_h = 256
model = models.Sequential([
              layers.Conv2D(32, 3, padding= 'same' , activation= 'relu', input_shape = (28, 28, 1) ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(32, 3 , padding= 'same', activation = 'relu',  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.3)),

              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.4)),

              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.5)),

              layers.Flatten(),
              layers.Dense(n_h, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Dense(n_h//2, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.Dense(10)
              ])

#model.build((None, 784))  # optional -- note None for the batch axis!!

opt = tf.optimizers.Adam(learning_rate= 5 * 1e-3)
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
def gradient(img_batch, lbl_batch):
  with tf.GradientTape() as tape:
      logits = model(img_batch, training=True)
        # loss format is generally: first argument targets, second argument outputs
      xent = loss_fn(lbl_batch, logits)

  varis = model.trainable_variables
  grads = tape.gradient(xent, varis)
  opt.apply_gradients(zip(grads, varis))
  return logits,xent

In [134]:
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
total_start = time.time()
for epoch in range(epochs):
  start = time.time()
  print("epoch number :", epoch)
  for step , (img_batch, lbl_batch) in enumerate(data):
    logits,xent = gradient(img_batch, lbl_batch)
    train_acc_metric(lbl_batch, logits)
    
  print("Loss: {} Accuracy: {}".format(xent, train_acc_metric.result()))
  train_acc_metric.reset_states()
  stop = time.time()
  print("took {} seconds\n".format(stop-start))
total_stop = time.time()
print("took {} seconds\n".format(total_stop-total_start))

epoch number : 0
Loss: 0.3062203824520111 Accuracy: 0.8267999887466431
took 29.65989637374878 seconds

epoch number : 1
Loss: 0.3652175962924957 Accuracy: 0.8876333236694336
took 29.68651247024536 seconds

epoch number : 2
Loss: 0.1978093385696411 Accuracy: 0.8990333080291748
took 29.489609479904175 seconds

epoch number : 3
Loss: 0.22875304520130157 Accuracy: 0.9081666469573975
took 29.466538429260254 seconds

epoch number : 4
Loss: 0.1916522979736328 Accuracy: 0.9141499996185303
took 29.505297899246216 seconds

epoch number : 5
Loss: 0.18503062427043915 Accuracy: 0.9180999994277954
took 29.45931911468506 seconds

epoch number : 6
Loss: 0.16792505979537964 Accuracy: 0.9205333590507507
took 29.673495054244995 seconds

epoch number : 7
Loss: 0.18069429695606232 Accuracy: 0.9257500171661377
took 29.50028133392334 seconds

epoch number : 8
Loss: 0.15301354229450226 Accuracy: 0.9273999929428101
took 29.49857258796692 seconds

epoch number : 9
Loss: 0.1370074301958084 Accuracy: 0.9286833405

In [135]:
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
  test_acc_metric(lbl_batch, model(img_batch))
print("Test acc: {}".format(test_acc_metric.result()))

Test acc: 0.9221000075340271


# Computation graphs

In [0]:
@tf.function
def gradient(img_batch, lbl_batch):
  with tf.GradientTape() as tape:
      logits = model(img_batch, training=True)
        # loss format is generally: first argument targets, second argument outputs
      xent = loss_fn(lbl_batch, logits)

  varis = model.trainable_variables
  grads = tape.gradient(xent, varis)
  opt.apply_gradients(zip(grads, varis))
  return logits,xent
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

In [0]:
# tf.enable_eager_execution()

fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
data = tf.data.Dataset.from_tensor_slices(
    (train_images.astype(np.float32) / 255.0, train_labels.astype(np.int32)))
data = data.batch(128)

# note: we batch the test data, but do not shuffle/repeat
test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.astype(np.float32) / 255.0, test_labels.astype(np.int32))).batch(128)

In [0]:
epochs = 10
n_h = 256
model = models.Sequential([
              layers.Conv2D(32, 3, padding= 'same' , activation= 'relu', input_shape = (28, 28, 1) ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(32, 3 , padding= 'same', activation = 'relu',  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.3)),

              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(64, 3 , padding= 'same', activation = 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.4)),

              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Conv2D(128, 3, padding= 'same', activation= 'relu' ,  kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.MaxPooling2D(),
              layers.Dropout((0.5)),

              layers.Flatten(),
              layers.Dense(n_h, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.BatchNormalization(),
              layers.Dense(n_h//2, activation='relu' , kernel_initializer= 'he_uniform'),
              layers.Dense(10)
              ])

#model.build((None, 784))  # optional -- note None for the batch axis!!
opt = tf.optimizers.Adam(learning_rate= 5 * 1e-3)
# from_logits = True!! #neverforget
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [156]:
# train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for step , (img_batch, lbl_batch) in enumerate(data):
  tf.summary.trace_on(graph=True, profiler=True)
  logits,xent = gradient(img_batch, lbl_batch)
  # train_acc_metric(lbl_batch, logits)
  with writer.as_default():
    tf.summary.trace_export(name="my_func_trace",step=0,profiler_outdir=logdir)
  # tf.print("Loss: {} Accuracy: {}".format(xent, train_acc_metric.result()))
  # train_acc_metric.reset_states()

In [0]:
# # Load the TensorBoard notebook extension.
# %reload_ext tensorboard

In [0]:
# %tensorboard --logdir logs/func

# DenseNet

In [0]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
# tf.keras.backend.clear_session()

In [0]:
# Setting up the hyper parameters
batch_size = 128
number_classes = 10
l = 5  #growth rate
number_filters = 36
compression = 0.5
dropout = 0.2

In [0]:
# Load the data
(train_images,train_labels) , (test_images,test_labels) = cifar10.load_data()
image_width = train_images.shape[1]
image_height = train_images.shape[2]
number_channels = train_images.shape[3]

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes= number_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes= number_classes)

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def dense_block(input, number_filters = 12, dropout = 0.2, scope = ''):
  x = input
  for i in range(1, l+1):

    # BN and Activation for last layer
    BatchNorm_BN = BatchNormalization(name=scope+'_layer_'+ str(i-1) +'_BN_'+str(i-1))(x)    
    relu_BN = Activation('relu', name=scope+'_layer_'+ str(i-1) +'_Relu_'+str(i-1))(BatchNorm_BN)  
    
    # Applying BottleNeck layer
    Conv2D_1 = Conv2D(int(number_filters), 1, padding='same',kernel_initializer='he_uniform', name=scope+'_BotNec_'+str(i))(relu_BN)
    BatchNorm = BatchNormalization(name=scope+'_BotNec_BN_'+str(i))(Conv2D_1)
    relu = Activation('relu', name=scope+'_BotNec_Relu_'+str(i))(BatchNorm)

    # Applying Conv layer
    Conv2D_3 = Conv2D(int(number_filters), 3, padding='same', kernel_initializer='he_uniform', name=scope+'_layer_'+str(i))(relu)
    # Conv2D_3 = Dropout(dropout, name=scope+'_Layer_'+ str(i) +'_Drop_'+str(i))(Conv2D_3)
    
    concat = Concatenate(axis=-1)([x,Conv2D_3])
    x = concat
  return x

In [0]:
def transition_layer(input,number_filters = 12,dropout = 0.2, scope = ''):
  BatchNorm = BatchNormalization(name=scope+'_BN')(input)
  relu = Activation('relu', name=scope+"_Relu")(BatchNorm)
  Conv2D_TL = Conv2D(int(number_filters * compression), 1, padding='same', kernel_initializer='he_uniform', name=scope+'_Conv')(relu)
  # Conv2D_TL = Dropout(dropout, name=scope+'_Drop')(Conv2D_TL)
  avg_pool = AveragePooling2D(pool_size=(2,2),strides=(2,2), name=scope+'_AVG_POOL')(Conv2D_TL)
  return avg_pool

In [0]:
def output_layer(input, scope):
  BatchNorm = BatchNormalization(name=scope+'_BN')(input)
  relu = Activation('relu', name=scope+'_Relu')(BatchNorm)
  avg_pool = AveragePooling2D(pool_size=(2,2), name=scope+'_AVG_POOL')(relu)
  temp = Conv2D(number_classes, 2, name=scope+'_Conv')(avg_pool)
  output = Activation('softmax', name=scope+'_Softmax')(temp)
  Flat = Flatten(name=scope+'_Flat')(output)
  return Flat


In [0]:
input = Input(shape = (image_height,image_width,number_channels), name='Input')
First_Conv2D = Conv2D(number_filters, 3, padding='same', name="Conv_0")(input)

First_Block = dense_block(First_Conv2D, number_filters, dropout, scope='DB_1')
First_Transition = transition_layer(First_Block, number_filters, dropout, scope='TB_1')

Second_Block = dense_block(First_Transition, number_filters, dropout, scope='DB_2')
Second_Transition = transition_layer(Second_Block, number_filters, dropout, scope='TB_2')

Third_Block = dense_block(Second_Transition, number_filters, dropout, scope='DB_3')
Third_Transition = transition_layer(Third_Block, number_filters, dropout, scope='TB_3')

Final_Block = dense_block(Third_Transition, number_filters, dropout, scope='FB')
output = output_layer(Final_Block, scope='Output')


In [0]:
model = Model(inputs = [input] , outputs = [output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv_0 (Conv2D)                 (None, 32, 32, 36)   1008        Input[0][0]                      
__________________________________________________________________________________________________
DB_1_layer_0_BN_0 (BatchNormali (None, 32, 32, 36)   144         Conv_0[0][0]                     
__________________________________________________________________________________________________
DB_1_layer_0_Relu_0 (Activation (None, 32, 32, 36)   0           DB_1_layer_0_BN_0[0][0]          
______________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(train_images)

Using TensorFlow backend.


In [0]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.005),metrics=['accuracy'])

In [0]:
model.fit(datagen.flow(train_images, train_labels, batch_size), steps_per_epoch = 3*train_images.shape[0]/batch_size, 
                    epochs = 40 ,validation_data =(test_images, test_labels))

Epoch 1/40
1172/1171 [==============================] - 293s 250ms/step - loss: 1.4622 - accuracy: 0.4778 - val_loss: 1.6217 - val_accuracy: 0.5207
Epoch 2/40
1172/1171 [==============================] - 291s 248ms/step - loss: 0.9266 - accuracy: 0.6715 - val_loss: 1.3761 - val_accuracy: 0.6036
Epoch 3/40
1172/1171 [==============================] - 290s 247ms/step - loss: 0.7222 - accuracy: 0.7459 - val_loss: 1.0623 - val_accuracy: 0.6877
Epoch 4/40
1172/1171 [==============================] - 289s 247ms/step - loss: 0.6046 - accuracy: 0.7895 - val_loss: 0.6625 - val_accuracy: 0.7786
Epoch 5/40
1172/1171 [==============================] - 289s 247ms/step - loss: 0.5349 - accuracy: 0.8133 - val_loss: 0.6126 - val_accuracy: 0.7968
Epoch 6/40
1172/1171 [==============================] - 290s 247ms/step - loss: 0.4798 - accuracy: 0.8324 - val_loss: 0.6565 - val_accuracy: 0.7932
Epoch 7/40
1172/1171 [==============================] - 290s 247ms/step - loss: 0.4398 - accuracy: 0.8475 - val_

In [0]:
model.evaluate(test_images,test_labels)

313/313 [==============================] - 7s 21ms/step - loss: 0.4763 - accuracy: 0.8788


[0.4762791395187378, 0.8787999749183655]

* We have build network with 4 dense blocks and 3 transition layers and then one output layer. With growth rate equal to 5 , learning rate as 0.005 ,no dropout and with data augmentation we were able to achieve test accuracy of 90.21% at 35th epoch. 
* Where as with growth rate as 8 , learning rate as 0.001, with dropout as 0.4 and without data augmentation test accuracy was around 86.7% at 23rd epoch.
* If we increase growth rate to 12, we might get better test accuracy.








